In [11]:
from mobo.surrogate_model import BoTorchSurrogateModel
from mobo.solver.botroch_solver import qNEHVISolver
from mobo.surrogate_problem import SurrogateProblem

from problems.common import build_problem

import numpy as np

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

train_x = np.random.uniform(8, 20, 15) 
train_x_2 = np.random.uniform(8, 20, 15)

train_y = 0.1 * train_x ** 2 - 3.5*train_x + np.cos(train_x*1.3) + 40 + np.random.normal(0, 0.01, len(train_x)) * train_x**2 * 0.3
train_y_2 = 0.1 * train_x_2 ** 2 - 3.5*train_x_2 + np.cos(train_x_2*1.3) + 40 + np.random.normal(0, 0.01, len(train_x_2)) * train_x_2**2 * 0.3

train_x = np.stack([train_x, train_x_2], axis=1)
train_y = np.stack([train_y, train_y_2], axis=1)

# Create some "gap" in the data
# to test epistemic uncertainty
# estimation:
gap_mask = np.array([(train_x[:,0]<12) | (train_x[:,0]>16), (train_x_2<12) | (train_x_2>16)]).T
train_x = train_x[gap_mask]
train_y = train_y[gap_mask]

# Scaling:
# train_x = (train_x - train_x.min()) / (train_x.max() - train_x.min())
# train_y = (train_y - train_y.mean()) / train_y.std()

# Scaling but keep dimensions:
train_x = (train_x - train_x.min(axis=0)) / (train_x.max(axis=0) - train_x.min(axis=0))
train_y = (train_y - train_y.mean(axis=0)) / train_y.std(axis=0)

# Plot training data:
plt.plot(train_x, train_y, 'k*')

import torch
import pyro
import gpytorch

class HeteroskedasticGP(gpytorch.models.ApproximateGP):
    
    def __init__(self, num_inducing=64, name_prefix="heteroskedastic_gp"):
        
        self.name_prefix = name_prefix

        # Define all the variational stuff
        inducing_points_task_one = torch.linspace(-1, 1, num_inducing)
        inducing_points_task_two = torch.linspace(-1, 1, num_inducing)
        
        inducing_points = torch.stack([inducing_points_task_one.unsqueeze(1), 
                                       inducing_points_task_two.unsqueeze(1)])
        
        # We have to mark the CholeskyVariationalDistribution as batch
        # so that we learn a variational distribution for each task (we have 2):
        variational_distribution = gpytorch.variational.CholeskyVariationalDistribution(
            num_inducing_points=inducing_points.size(-2), 
            batch_shape=torch.Size([2]))
        
        single_variational_strategy = gpytorch.variational.VariationalStrategy(
                self, inducing_points, variational_distribution, learn_inducing_locations=True)
        
        # Wrap the single variational strategy into a
        # Linear Model of Coregionalization one, so the two
        # tasks (and therefore latent GPs) are assumed to be
        # somehow related (and we learn such relationship):
        variational_strategy = gpytorch.variational.LMCVariationalStrategy(single_variational_strategy,
                                                                           num_tasks=2,
                                                                           num_latents=2)
        # Standard initializtation
        super().__init__(variational_strategy)
        
        # The mean and covariance modules should be marked as batch
        # so we learn a different set of hyperparameters:
        self.mean_module = gpytorch.means.ConstantMean(batch_shape=torch.Size([2]))
        
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel(batch_shape=torch.Size([2])),
            batch_shape=torch.Size([2])
        )

    def forward(self, x):
        # The forward function should be written as if we were dealing with each output
        # dimension in batch
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

    def guide(self, x, y):
        
        # Get q(f) - variational (guide) distribution of latent function
        function_dist = self.pyro_guide(x)

        # Use a plate here to mark conditional independencies.
        # Our samples are of shape (nobs, 2), therefore the dim=-2
        # in the plate:
        with pyro.plate(self.name_prefix + ".data_plate", dim=-2):
            
            # Sample from latent function distribution
            function_samples = pyro.sample(self.name_prefix + ".f(x)", function_dist)

    def model(self, x, y):
        
        pyro.module(self.name_prefix + ".gp", self)

        # Get p(f) - prior distribution of latent function
        function_dist = self.pyro_model(x)

        # Use a plate here to mark conditional independencies.
        # Our samples are of shape (nobs, 2), therefore the dim=-2
        # in the plate:
        with pyro.plate(self.name_prefix + ".data_plate", dim=-2):
            
            # Sample from latent function distribution
            function_samples = pyro.sample(self.name_prefix + ".f(x)", function_dist)
            
            mean_samples = function_samples[...,0]
            std_samples = function_samples[...,1]
            
            # Exp to force always nonnegative stddevs:
            transformed_std_samples = torch.exp(std_samples)

            # Sample from observed distribution
            return pyro.sample(self.name_prefix + ".y",
                               pyro.distributions.Normal(mean_samples, transformed_std_samples),
                               obs=y)

# Instantiate model:
pyro.clear_param_store()  # Good practice
model = HeteroskedasticGP()

num_iter = 1000
num_particles = 256

# training routine:
def train():
    optimizer = pyro.optim.Adam({"lr": 0.01})
    elbo = pyro.infer.Trace_ELBO(num_particles=num_particles, vectorize_particles=True, retain_graph=True)
    svi = pyro.infer.SVI(model.model, model.guide, optimizer, elbo)

    model.train()
    iterator = range(num_iter)
    for i in iterator:
        model.zero_grad()
        loss = svi.step(torch.from_numpy(train_x).float().unsqueeze(1), torch.from_numpy(train_y).float())
        print(loss)

# Train the GP:
train()

# Function to plot predictions:
def plot_preds(ax, feature, mean_samples, var_samples, bootstrap=True, n_boots=100,
               show_epistemic=False, epistemic_mean=None, epistemic_var=None):
    """Plots the overall mean and variance of the aggregate system.
    Inherited from https://www.pymc.io/projects/examples/en/latest/gaussian_processes/GP-Heteroskedastic.html.

    We can represent the overall uncertainty via explicitly sampling the underlying normal
    distributrions (with `bootstrap=True`) or as the mean +/- the standard deviation from
    the Law of Total Variance. 
    
    For systems with many observations, there will likely be
    little difference, but in cases with few observations and informative priors, plotting
    the percentiles will likely give a more accurate representation.
    """
    if bootstrap:
        means = np.expand_dims(mean_samples.T, axis=2)
        stds = np.sqrt(np.expand_dims(var_samples.T, axis=2))
        
        samples_shape = mean_samples.T.shape + (n_boots,)
        samples = np.random.normal(means, stds, samples_shape)
        
        reshaped_samples = samples.reshape(mean_samples.shape[1], -1).T
        
        l, m, u = [np.percentile(reshaped_samples, p, axis=0) for p in [2.5, 50, 97.5]]
        ax.plot(feature, m, label="Median", color="b")
    
    else:
        m = mean_samples.mean(axis=0)
        sd = np.sqrt(mean_samples.var(axis=0) + var_samples.mean(axis=0))
        l, u = m - 1.96 * sd, m + 1.96 * sd
        ax.plot(feature, m, label="Mean", color="b")
    
    if show_epistemic:
        ax.fill_between(feature,
                        l,
                        epistemic_mean-1.96*np.sqrt(epistemic_var), 
                        alpha=0.2,
                        color="#88b4d2",
                        label="Total Uncertainty (95%)")
        ax.fill_between(feature, 
                        u, 
                        epistemic_mean+1.96*np.sqrt(epistemic_var), 
                        alpha=0.2,
                        color="#88b4d2")
        ax.fill_between(feature, 
                        epistemic_mean-1.96*np.sqrt(epistemic_var), 
                        epistemic_mean+1.96*np.sqrt(epistemic_var),
                        alpha=0.4,
                        color="#88b4d2",
                        label="Epistemic Uncertainty (95%)")
    else:
        ax.fill_between(feature, 
                        l, 
                        u, 
                        alpha=0.2,
                        color="#88b4d2",
                        label="Total Uncertainty (95%)")

# Test data to predict:
x_padding = 0.1

test_x = torch.linspace(train_x.min() - (train_x.max() - train_x.min()) * x_padding, 
                        train_x.max() + (train_x.max() - train_x.min()) * x_padding, 
                        100).float()

# Predict:
model.eval()
with torch.no_grad():
    output_dist = model(test_x)

# Extract predictions:
output_samples = output_dist.sample(torch.Size([1000]))
mu_samples = output_samples[...,0]
sigma_samples = torch.exp(output_samples[...,1])

# Plot predictions:
plt.plot(train_x, train_y, 'k*', label="Observed Data")
ax = plt.gca()
plot_preds(ax, 
           test_x.numpy(),
           mu_samples.numpy(), 
           sigma_samples.numpy()**2,
           bootstrap=False, 
           n_boots=100,
           show_epistemic=True,
           epistemic_mean=output_dist.mean[:,0].detach().numpy(),
           epistemic_var=output_dist.stddev[:,0].detach().numpy()**2)
ax.legend();


c:\Users\queim\micromambaenv\envs\mobo\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


56.905513763427734
43.140201568603516
33.97520065307617
32.39889907836914
30.88536262512207
30.37556266784668
30.333194732666016
30.811912536621094
30.840333938598633
30.82563591003418
30.904239654541016
30.56078338623047
30.418033599853516
30.40366554260254
29.947216033935547
29.68463134765625
29.23786163330078
29.205806732177734
28.808366775512695
28.628103256225586
28.46971893310547
27.884201049804688
27.8964786529541
27.303796768188477
27.3316650390625
26.636533737182617
26.283199310302734
26.21200180053711
25.863445281982422
25.856975555419922
25.515331268310547
25.21506118774414
25.08490753173828
24.485965728759766
24.269908905029297
23.99302864074707
23.758787155151367
23.821086883544922
23.71884536743164
22.92620277404785
22.89940643310547
23.036392211914062
22.452402114868164
22.066085815429688
22.594179153442383
21.929698944091797
21.58176612854004
21.861177444458008
21.51170539855957
21.30780601501465
21.10140609741211
20.818603515625
20.646430015563965
20.489727020263672
20

In [ ]:

def initialize_model(train_x, train_y, train_rho=None):
    # define models for objective and constraint
    train_y_mean = -train_y  # negative because botorch assumes maximization
    # train_y_var = self.real_problem.evaluate(train_x).to(**tkwargs).var(dim=-1)
    train_y_var = train_rho + 1e-6
    models = []
    for i in range(train_y.shape[1]):
        train_y_i = train_y_mean[..., i]
        train_yvar_i = train_y_var[..., i]
        # models.append(
        #     FixedNoiseGP(
        #         train_X=train_x,
        #         train_Y=train_y_i.unsqueeze(-1),
        #         train_Yvar=train_yvar_i.unsqueeze(-1),
        #         outcome_transform=Standardize(m=1),
        #     )
        # )
        models.append(
            HeteroskedasticSingleTaskGP(
                train_X=train_x,
                train_Y=train_y_i.unsqueeze(-1),
                train_Yvar=train_yvar_i.unsqueeze(-1),
                outcome_transform=Standardize(m=1),
            )
        )
        # models.append(
        #     SingleTaskGP(
        #         train_x,
        #         train_y_i.unsqueeze(-1),
        #         # train_yvar_i.unsqueeze(-1),
        #         outcome_transform=Standardize(m=1),
        #     )
        # )
    model = ModelListGP(*models)
    mll = SumMarginalLogLikelihood(model.likelihood, model)

    return mll, model